# Capstone

### Fernando Díaz Loo

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Wikipedia

In [2]:
website_url = requests.get("https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1").text
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="es">
 <head>
  <meta charset="utf-8"/>
  <title>
   Anexo:Localidades de Bogotá - Wikipedia, la enciclopedia libre
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Anexo","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":104,"wgPageName":"Anexo:Localidades_de_Bogotá","wgTitle":"Localidades de Bogotá","wgCurRevisionId":114573108,"wgRevisionId":114573108,"wgArticleId":6319494,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Localidades de Bogotá","Anexos:Bogotá"],"wgBreakFrames":false,"wgPageContentLanguage":"es","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMont

In [3]:
My_table = soup.find('table',{'class':'sortable wikitable'})
My_table

<table border="1" class="sortable wikitable">
<tbody><tr bgcolor="silver">
<th width="23"><center>Nº
</center></th>
<th width="110"><center> Localidad </center>
</th>
<th width="90"><center> Códigos Postales </center>
</th>
<th><center> Superficie km²<sup class="reference separada" id="cite_ref-2"><a href="#cite_note-2"><span class="corchete-llamada">[</span>2<span class="corchete-llamada">]</span></a></sup>​</center>
</th>
<th><center> Población<sup class="reference separada" id="cite_ref-3"><a href="#cite_note-3"><span class="corchete-llamada">[</span>3<span class="corchete-llamada">]</span></a></sup>​</center>
</th>
<th><center> Densidad hab/km²</center>
</th></tr>
<tr>
<td><b>01</b>
</td>
<td><a href="/wiki/Usaqu%C3%A9n" title="Usaquén">Usaquén</a>
</td>
<td>11<b>01</b>11-11<b>01</b>51
</td>
<td>65.31
</td>
<td>501 999
</td>
<td>7 686.4
</td></tr>
<tr>
<td><b>02</b>
</td>
<td><a href="/wiki/Chapinero" title="Chapinero">Chapinero</a>
</td>
<td>11<b>02</b>11-11<b>02</b>31
</td>
<td>3

In [4]:
links = My_table.findAll('a')
links

[<a href="#cite_note-2"><span class="corchete-llamada">[</span>2<span class="corchete-llamada">]</span></a>,
 <a href="#cite_note-3"><span class="corchete-llamada">[</span>3<span class="corchete-llamada">]</span></a>,
 <a href="/wiki/Usaqu%C3%A9n" title="Usaquén">Usaquén</a>,
 <a href="/wiki/Chapinero" title="Chapinero">Chapinero</a>,
 <a href="/wiki/Santa_Fe_(Bogot%C3%A1)" title="Santa Fe (Bogotá)">Santa Fe</a>,
 <a href="/wiki/San_Crist%C3%B3bal_(Bogot%C3%A1)" title="San Cristóbal (Bogotá)">San Cristóbal</a>,
 <a href="/wiki/Usme" title="Usme">Usme</a>,
 <a href="/wiki/Tunjuelito" title="Tunjuelito">Tunjuelito</a>,
 <a href="/wiki/Bosa_(Bogot%C3%A1)" title="Bosa (Bogotá)">Bosa</a>,
 <a href="/wiki/Kennedy_(Bogot%C3%A1)" title="Kennedy (Bogotá)">Kennedy</a>,
 <a href="/wiki/Fontib%C3%B3n" title="Fontibón">Fontibón</a>,
 <a href="/wiki/Engativ%C3%A1" title="Engativá">Engativá</a>,
 <a href="/wiki/Suba" title="Suba">Suba</a>,
 <a href="/wiki/Barrios_Unidos_(Bogot%C3%A1)" title="Barrios 

In [5]:
Localidades = []
for link in links:
    Localidades.append(link.get('title'))
    
print(Localidades)

del Localidades[0:2]

Localidades

[None, None, 'Usaquén', 'Chapinero', 'Santa Fe (Bogotá)', 'San Cristóbal (Bogotá)', 'Usme', 'Tunjuelito', 'Bosa (Bogotá)', 'Kennedy (Bogotá)', 'Fontibón', 'Engativá', 'Suba', 'Barrios Unidos (Bogotá)', 'Teusaquillo', 'Los Mártires', 'Antonio Nariño (Bogotá)', 'Puente Aranda', 'La Candelaria', 'Rafael Uribe Uribe (Bogotá)', 'Ciudad Bolívar (Bogotá)', 'Sumapaz (Bogotá)']


['Usaquén',
 'Chapinero',
 'Santa Fe (Bogotá)',
 'San Cristóbal (Bogotá)',
 'Usme',
 'Tunjuelito',
 'Bosa (Bogotá)',
 'Kennedy (Bogotá)',
 'Fontibón',
 'Engativá',
 'Suba',
 'Barrios Unidos (Bogotá)',
 'Teusaquillo',
 'Los Mártires',
 'Antonio Nariño (Bogotá)',
 'Puente Aranda',
 'La Candelaria',
 'Rafael Uribe Uribe (Bogotá)',
 'Ciudad Bolívar (Bogotá)',
 'Sumapaz (Bogotá)']

In [6]:
df = pd.DataFrame()
df['Localidades'] = Localidades
df

,Localidades
0,Usaquén
1,Chapinero
2,Santa Fe (Bogotá)
3,San Cristóbal (Bogotá)
4,Usme
5,Tunjuelito
6,Bosa (Bogotá)
7,Kennedy (Bogotá)
8,Fontibón
9,Engativá


In [7]:
def get_coords_local(localidad, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name

    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+', Bogota, Bogota Capital District',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [8]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Usaquén
Chapinero
Santa Fe (Bogotá)
San Cristóbal (Bogotá)
Usme
Tunjuelito
Bosa (Bogotá)
Kennedy (Bogotá)
Fontibón
Engativá
Suba
Barrios Unidos (Bogotá)
Teusaquillo
Los Mártires
Antonio Nariño (Bogotá)
Puente Aranda
La Candelaria
Rafael Uribe Uribe (Bogotá)
Ciudad Bolívar (Bogotá)
Sumapaz (Bogotá)


(20, 3)

In [9]:
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Bogotá are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Bogotá are 4.5980772, -74.0761028.


In [10]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

,Localidades,Latitude,Longitude
0,Usaquén,4.695047,-74.031493
1,Chapinero,4.647120,-74.063458
2,Santa Fe (Bogotá),4.593766,-74.034314
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.411136,-74.129108
5,Tunjuelito,4.560148,-74.128922
6,Bosa (Bogotá),4.625492,-74.200280
7,Kennedy (Bogotá),4.629682,-74.149935
8,Fontibón,4.678737,-74.146988
9,Engativá,4.696628,-74.106120


In [11]:
# create map of Bogotá using latitude and longitude values
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bogota)  
    
map_bogota

In [19]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [21]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'ZMHWBS0SR12Z3YDYVHJVTZPRK3U1ZP3I2TYQAJ5CU3JUHMB5'
CLIENT_SECRET = 'H3TT0XT3P5TIAFCV1Y2UUVLF42N44DICNKLUELK34H2TKLFR'
VERSION = '20181020'

In [22]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
bogota_venues_burger = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
bogota_venues_burger.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.695047,-74.031493,La Hamburguesería,4.695310,-74.031716,Burger Joint
1,Usaquén,4.695047,-74.031493,Home Burgers,4.693883,-74.032853,Burger Joint
2,Usaquén,4.695047,-74.031493,Burgers and Beers,4.695779,-74.029240,Burger Joint
3,Usaquén,4.695047,-74.031493,Martinica Motos,4.695222,-74.031873,Burger Joint
4,Usaquén,4.695047,-74.031493,El corral Gourmet,4.701353,-74.029992,Burger Joint


In [23]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [24]:
map_bogota_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_burger, 'red', map_bogota_burger)
map_bogota_burger

In [25]:
bogota_venues_highschools = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
bogota_venues_highschools.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.695047,-74.031493,Gimnasio Los Cerros,4.694464,-74.026556,School
1,Chapinero,4.647120,-74.063458,Colegio Nuestra Señora de Chiquinquira,4.649800,-74.065643,High School
2,Chapinero,4.647120,-74.063458,Colegio Distrital Simón Rodriguez,4.653948,-74.060732,High School
3,Chapinero,4.647120,-74.063458,Colegio del Sagrado Corazón de Jesus Bethlemitas,4.649182,-74.058036,High School
4,Chapinero,4.647120,-74.063458,Colegio Ervit,4.642634,-74.066469,High School


In [26]:
map_bogota_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_highschools, 'green', map_bogota_highschools)
map_bogota_highschools

In [27]:
bogota_venues_uni = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
bogota_venues_uni.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.695047,-74.031493,Tecnologico de Monterry,4.692100,-74.034916,University
1,Usaquén,4.695047,-74.031493,Centrum Católica,4.690426,-74.038657,University
2,Usaquén,4.695047,-74.031493,UTadeo M7a,4.705390,-74.032827,University
3,Chapinero,4.647120,-74.063458,CIDCA,4.647201,-74.063729,University
4,Chapinero,4.647120,-74.063458,Universidad Konrad Lorenz,4.648584,-74.061586,University


In [28]:
map_bogota_uni = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_uni, 'gold', map_bogota_uni)
map_bogota_uni

In [29]:
bogota_venues_office = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
bogota_venues_office.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.695047,-74.031493,UNIVERSIDAD DEL SINÚ,4.695240,-74.030510,General College & University
1,Usaquén,4.695047,-74.031493,Centro Médico de la Sabana,4.696026,-74.031596,Doctor's Office
2,Usaquén,4.695047,-74.031493,3HTP Colombia,4.694255,-74.032575,Office
3,Usaquén,4.695047,-74.031493,Fundacion Santa Fe de Bogota,4.695136,-74.033211,Hospital
4,Usaquén,4.695047,-74.031493,Santa Ana Medical Center,4.696085,-74.032065,Doctor's Office


In [30]:
map_bogota_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_office, 'fuchsia', map_bogota_office)
map_bogota_office

In [31]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [32]:
df_data = df3.copy()
df_data.rename(columns={'Localidades':'Localidad'}, inplace=True)
addColumn(df_data, 'Burger', bogota_venues_burger)
addColumn(df_data, 'High Schools', bogota_venues_highschools)
addColumn(df_data, 'Universities', bogota_venues_uni)
addColumn(df_data, 'Offices', bogota_venues_office)
df_data

,Localidad,Latitude,Longitude,Burger,High Schools,Universities,Offices
0,Usaquén,4.695047,-74.031493,22.0,1.0,3.0,50.0
1,Chapinero,4.647120,-74.063458,38.0,4.0,36.0,50.0
2,Santa Fe (Bogotá),4.593766,-74.034314,0.0,0.0,0.0,0.0
3,San Cristóbal (Bogotá),4.548658,-74.047473,0.0,0.0,0.0,0.0
4,Usme,4.411136,-74.129108,0.0,0.0,0.0,0.0
5,Tunjuelito,4.560148,-74.128922,2.0,2.0,0.0,44.0
6,Bosa (Bogotá),4.625492,-74.200280,0.0,1.0,0.0,22.0
7,Kennedy (Bogotá),4.629682,-74.149935,12.0,1.0,1.0,46.0
8,Fontibón,4.678737,-74.146988,4.0,3.0,2.0,46.0
9,Engativá,4.696628,-74.106120,4.0,5.0,1.0,48.0


In [33]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_burger = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_uni = 1.5

# positive weight because employees are even better customers
weight_offices = 2

In [34]:
df_weighted = df_data[['Localidad']].copy()

In [35]:
df_weighted['Score'] = df_data['Burger'] * weight_burger + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Localidad,Score
16,La Candelaria,130.5
1,Chapinero,120.0
12,Teusaquillo,102.5
14,Antonio Nariño (Bogotá),101.0
9,Engativá,98.5
13,Los Mártires,95.5
11,Barrios Unidos (Bogotá),94.5
15,Puente Aranda,94.0
8,Fontibón,94.0
17,Rafael Uribe Uribe (Bogotá),92.0


In [36]:
map_bog_result = folium.Map(location=[latitude, longitude], zoom_start=15)

bog_win = df3[df3['Localidades'] == 'La Candelaria']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], bog_win['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_bog_result) 

addToMap(bogota_venues_burger[bogota_venues_burger['Localidad'] == 'La Candelaria'], 'red', map_bog_result)
addToMap(bogota_venues_highschools[bogota_venues_highschools['Localidad'] == 'La Candelaria'], 'green', map_bog_result)
addToMap(bogota_venues_uni[bogota_venues_uni['Localidad'] == 'La Candelaria'], 'gold', map_bog_result)
addToMap(bogota_venues_office[bogota_venues_office['Localidad'] == 'La Candelaria'], 'fuchsia', map_bog_result)

map_bog_result

In [ ]:
### La Candelaria is the best option for the Burger Shop